In [1]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.6/769.6 kB 63.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [3]:
# Download stopwords and punkt tokenizer from NLTK
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /home/wsuser/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/wsuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='gQHGPKfSS9ra4Tie0SYCom5XBBjih7GficmCX-V2opao',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu-de.cloud-object-storage.appdomain.cloud')

bucket = 'amazonmlchallenge-donotdelete-pr-oaqyh2hr4halsq'
object_key = 'train.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

train_df = pd.read_csv(body)
train_df.head()


,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650,2125.980000
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755,393.700000
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
4,283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112,598.424000


In [6]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='gQHGPKfSS9ra4Tie0SYCom5XBBjih7GficmCX-V2opao',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu-de.cloud-object-storage.appdomain.cloud')

bucket = 'amazonmlchallenge-donotdelete-pr-oaqyh2hr4halsq'
object_key = 'test.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

test_df = pd.read_csv(body)
test_df.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID
0,604373,Manuel d'Héliogravure Et de Photogravure En Re...,NaN,NaN,6142
1,1729783,DCGARING Microfiber Throw Blanket Warm Fuzzy P...,[QUALITY GUARANTEED: Luxury cozy plush polyest...,<b>DCGARING Throw Blanket</b><br><br> <b>Size ...,1622
2,1871949,I-Match Auto Parts Front License Plate Bracket...,"[Front License Plate Bracket Made Of Plastic,D...",Replacement for The Following Vehicles:2020 LE...,7540
3,1107571,PinMart Gold Plated Excellence in Service 1 Ye...,[Available as a single item or bulk packed. Se...,Our Excellence in Service Lapel Pins feature a...,12442
4,624253,"Visual Mathematics, Illustrated by the TI-92 a...",NaN,NaN,6318


In [7]:
# Define a function to preprocess the text data
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # Stem the tokens
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    # Join the stemmed tokens back into a string
    preprocessed_text = ' '.join(stemmed_tokens)
    return preprocessed_text

In [8]:
def preprocess_text(text):
    if isinstance(text, float) and np.isnan(text):
        return ''
    # Tokenize the text
    words = word_tokenize(text.lower())
    
    # Remove stopwords
    words = [word for word in words if word not in stop_words]
    
    # Join the remaining words
    return ' '.join(words)

In [9]:
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
def preprocess_text(text):
    """
    Removes leading and trailing whitespace from the given text string.

    Parameters:
    text (str or float): The text string to preprocess.

    Returns:
    str or float: The preprocessed text string with leading and trailing whitespace removed, or the original input if it is not a string.
    """
    if isinstance(text, str):
        return text.strip()
    else:
        return text


# Combine the text columns and preprocess the text
train_df['TEXT'] = train_df['TITLE'] + ' ' + train_df['DESCRIPTION'] + ' ' + train_df['BULLET_POINTS']
train_df['TEXT'] = train_df['TEXT'].apply(preprocess_text)
test_df['TEXT'] = test_df['TITLE'] + ' ' + test_df['DESCRIPTION'] + ' ' + test_df['BULLET_POINTS']
test_df['TEXT'] = test_df['TEXT'].apply(preprocess_text)

In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_df['TEXT'], train_df['PRODUCT_LENGTH'], test_size=0.2, random_state=42)

In [ ]:
# Replace missing values with empty strings
X_train = X_train.fillna('')
X_val = X_val.fillna('')

# Extract features from the text columns using TfidfVectorizer
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)


In [ ]:
# Train a linear regression model
reg = LinearRegression()
reg.fit(X_train_tfidf, y_train)


In [ ]:
# Make predictions on the validation set
y_pred = reg.predict(X_val_tfidf)

In [ ]:
from sklearn import metrics
# Evaluate the model using mean absolute percentage error
score = max(0,100*(1-metrics.mean_absolute_percentage_error(y_val, y_pred)))
print("Model score: ", score)

In [ ]:
test_df['TEXT'] = test_df['TEXT'].fillna('')
# Make predictions on the test set
test_df_tfidf = tfidf.transform(test_df['TEXT'])
test_df['PRODUCT_LENGTH'] = reg.predict(test_df_tfidf)


In [ ]:
from project_lib import Project
import pandas as pd

# Set up the project object
project = Project(None, "e149480b-6547-4478-8897-98a96ce36d15", "p-eee9223a6d49755e4097f9449708b24d17c6b86c")

# Create a Pandas dataframe with your data
submission_df = test_df[['PRODUCT_ID', 'PRODUCT_LENGTH']]

# Save the dataframe to a CSV file in IBM Watson Studio
project.save_data("submission.csv", submission_df.to_csv(index=False), overwrite=True)